In [12]:
import pcl
import rosbag
import numpy as np
import gtsam as gt
import ros_numpy
import sensor_msgs
import time
from tqdm import tqdm

In [13]:
bag = rosbag.Bag('./data/2022-04-04-18-44-50.bag')

In [14]:
vertex = {} # {time: [i, gt.Pose2]}
i = int(0)
for vertex_str, vertex_msg, vertex_time in bag.read_messages(topics=['/vertex_odom']):
    temp = vertex_msg.data.split(' ')
    t,x,y,theta = np.array(temp[1:]).astype(np.float64)
    temp_pose = gt.Pose2(x, y, theta)
    vertex[t] = [i,temp_pose]
    i = int(i+1)

In [15]:
edges = [] # i,j,x,y,theta,noiseModel
for edge_str, edge_msg, edge_time in bag.read_messages(topics=['/edge_odom']):
    data = edge_msg.data.split(' ')
    st,ct,x,y,theta = np.array(data[1:6]).astype(np.float64)
    q11, q12, q13, q22, q23, q33 = np.array(data[6:]).astype(np.float64)
    info = np.array([q11, q12, q13, q12, q22, q23, q13, q23, q33]).reshape([3,3])
    noiseModel = gt.noiseModel.Gaussian.Information(info)
    if vertex.get(st) is not None and vertex.get(ct) is not None:
        i = vertex.get(st)[0]
        j = vertex.get(ct)[0]
        edges.append([i,j,x,y,theta,noiseModel])

In [16]:
def convert2PCL(pc_msg):
    pc_msg.__class__ = sensor_msgs.msg._PointCloud2.PointCloud2
    offset_sorted = {f.offset: f for f in pc_msg.fields}
    pc_msg.fields = [f for (_, f) in sorted(offset_sorted.items())]
    pc_np = ros_numpy.point_cloud2.pointcloud2_to_xyz_array(pc_msg, remove_nans=True)
    pc_pcl = pcl.PointCloud(np.array(pc_np, dtype=np.float32))
    return pc_pcl

In [17]:
pc_dict = {} # {id: ros::pc2}
for pc_str, pc_msg, pc_time in bag.read_messages(topics=['/currentFeatureInWorld']):
    ct = pc_msg.header.stamp.to_sec()
    idx = vertex.get(ct, vertex[min(vertex.keys(), key=lambda k: abs(k-ct))])[0]  #Find closest index
    pc_dict[idx] = convert2PCL(pc_msg)

In [19]:
#https://github.com/strawlab/python-pcl/blob/master/examples/official/Registration/iterative_closest_point.py
fitnessThreshold =1.8
interEdges = [] # [i,j,T] 
for idx, inputCloud in tqdm(pc_dict.items()):
    icp = inputCloud.make_IterativeClosestPoint()
    matchMatrix = [] #[i,fitnessScore,T]
    for i, targetCloud in tqdm(pc_dict.items()):
        if i != idx:
            boolCoverge, T, estimate, fitnessScore = icp.icp(inputCloud, targetCloud)
            matchMatrix.append([i,fitnessScore,T])
    matchMatrix = np.array(matchMatrix)
    bestMatches = matchMatrix[matchMatrix[:,1]>fitnessThreshold]
    for bM in bestMatches:
        interEdges.append([idx, bM[0], bM[2]]) # [i,j,T]   

  0%|          | 0/2879 [00:00<?, ?it/s]


ValueError: order must be one of 'C', 'F', 'A', or 'K' (got 'fortran')

In [ ]:
np.savez('processedData.npz', vertex=**vertex, edges=np.array(edges), interEdges=np.array(interEdges))